###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Educación Nacional

In [1]:
import pandas as pd
import gc

###Archivo sty3-c395.csv

Contiene información sobre la matrícula estadística de Educación Preescolar Básica y Media de Colombia del corte 2018 al 2023.Agregado de matricula por establecimiento educativo, que contiene las variables del establecimiento (sede, zona, jornada, grado, metodología, carácter y sector CONPES) y de la matricula (edad, género y grupo étnico)

In [2]:
# Especificar las columnas que deseas cargar
columnas_necesarias = ['anno_inf','departamento', 'secretaria', 'cod_dane_municipio', 'municipio','codigo_dane',
       'nombre_establecimiento', 'sector', 'calendario', 'nombre_sede','zona','tipo_jornada', 'grado', 'metodologia', 'caracter',
        'especialidad', 'edad', 'genero', 'grupo_etnico', 'total_matricula' ]

# Lista para acumular los fragmentos de datos
data_chunks = []

# Leer el archivo en fragmentos (chunks) de 500,000 filas, solo cargando las columnas necesarias
chunksize = 500000
for chunk in pd.read_csv("https://www.datos.gov.co/resource/sty3-c395.csv?$limit=6200000&$offset=6200000",
                         dtype={'cod_dane_municipio': str},
                         chunksize=chunksize,
                         usecols=columnas_necesarias):

    # Procesar o modificar cada chunk si es necesario
    # Agregar el fragmento a la lista
    data_chunks.append(chunk)

# Concatenar todos los fragmentos en un solo DataFrame
df = pd.concat(data_chunks, ignore_index=True)

# Limpiar la memoria liberando la lista de fragmentos
del data_chunks
gc.collect()

# Mostrar las primeras filas del DataFrame Dask
df.head()

,anno_inf,departamento,secretaria,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,metodologia,caracter,especialidad,edad,genero,grupo_etnico,total_matricula
0,2022,VALLE DEL CAUCA,PALMIRA,76520,PALMIRA,276520000564,INSTITUCION EDUCATIVA TABLONES,OFICIAL,A,TABLONES,Rural,Fin de Semana,Ciclo 6 Adultos,Programa para jóvenes en extraedad y adultos,No Aplica,No aplica,19,Femenino,No aplica,2
1,2022,VALLE DEL CAUCA,CALI,76001,CALI,376001028529,COLEGIO SAN IGNACIO DE LOYOLA,NO_OFICIAL,B,COLEGIO SAN IGNACIO DE LOYOLA,Urbana,Mañana,Octavo,Educación tradicional,No Aplica,No aplica,15,Masculino,No aplica,1
2,2022,VALLE DEL CAUCA,CALI,76001,CALI,376001028529,COLEGIO SAN IGNACIO DE LOYOLA,NO_OFICIAL,B,COLEGIO SAN IGNACIO DE LOYOLA,Urbana,Mañana,Octavo,Educación tradicional,No Aplica,No aplica,14,Femenino,No aplica,2
3,2022,VALLE DEL CAUCA,CALI,76001,CALI,376001028529,COLEGIO SAN IGNACIO DE LOYOLA,NO_OFICIAL,B,COLEGIO SAN IGNACIO DE LOYOLA,Urbana,Mañana,Octavo,Educación tradicional,No Aplica,No aplica,13,Masculino,No aplica,2
4,2022,VALLE DEL CAUCA,CALI,76001,CALI,376001029908,LICEO JUVENIL EL RODEO,NO_OFICIAL,A,LICEO JUVENIL EL RODEO,Urbana,Mañana,Octavo,Educación tradicional,No Aplica,No aplica,13,Femenino,No aplica,3


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5955274 entries, 0 to 5955273
Data columns (total 20 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   anno_inf                int64 
 1   departamento            object
 2   secretaria              object
 3   cod_dane_municipio      object
 4   municipio               object
 5   codigo_dane             int64 
 6   nombre_establecimiento  object
 7   sector                  object
 8   calendario              object
 9   nombre_sede             object
 10  zona                    object
 11  tipo_jornada            object
 12  grado                   object
 13  metodologia             object
 14  caracter                object
 15  especialidad            object
 16  edad                    int64 
 17  genero                  object
 18  grupo_etnico            object
 19  total_matricula         int64 
dtypes: int64(4), object(16)
memory usage: 908.7+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
description_col = {
    "anno_inf": "Año del reporte de Información",
    "cod_dane_departamento": "Código del Departamento de la Sede",
    "departamento": "Departamento de la Sede",
    "cod_secretaria": "Código de la Secretaría de Educación donde ha sido atendido el estudiante",
    "secretaria": "Secretaría de Educación donde ha sido atendido el estudiante",
    "cod_dane_municipio": "Código del Municipio de la Sede",
    "municipio": "Municipio de la Sede",
    "codigo_dane": "Municipio Dane Establecimiento",
    "nombre_establecimiento": "Nombre del Establecimiento",
    "cod_sector": "Código del Sector de atención del Establecimiento",
    "sector": "Sector de atención del Establecimiento",
    "cod_calendario": "Código del Calendario del Establecimiento",
    "calendario": "Calendario del Establecimiento",
    "codigo_dane_sede": "Código Dane de la Sede",
    "nombre_sede": "Nombre de la Sede",
    "cod_zona": "Código de la Zona donde se encuentra la sede",
    "zona": "Zona donde se encuentra la sede",
    "cod_tipo_jornada": "Código del tipo de Jornada de la Sede",
    "tipo_jornada": "Tipo de Jornada de la Sede",
    "codigo_grado": "Código grado",
    "grado": "Descriptivo del grado",
    "cod_metodologia": "Código de la metodología",
    "metodologia": "Metodología",
    "cod_caracter": "Código del Carácter del Establecimiento",
    "caracter": "Carácter del Establecimiento",
    "cod_especialidad": "Código de la Especialidad del Establecimiento",
    "especialidad": "Especialidad del Establecimiento",
    "edad": "Edad calculada con referencia al inicio del calendario escolar",
    "cod_genero": "Letra que identifica el Sexo Biológico",
    "genero": "Sexo Biológico",
    "cod_grupo_etnico": "Código del Grupo Étnico",
    "grupo_etnico": "Grupo Étnico",
    "cod_sector_conpes": "Código del sector para asignación de recursos CONPES",
    "sector_conpes": "Sector para asignación de recursos CONPES",
    "total_matricula": "Total de Matrícula 2018"
}

In [5]:
# Dejar columnas importantes
relevant_cols = ['anno_inf','departamento', 'cod_dane_municipio', 'municipio','codigo_dane',
       'nombre_establecimiento', 'sector', 'calendario', 'nombre_sede','zona','tipo_jornada', 'grado',
        'especialidad', 'edad', 'genero', 'grupo_etnico', 'total_matricula' ]
# Crear nuevo dataframe dask con las columnas importantes
preschool_primary_secondary_education_Mineducacion_part_2 = df[relevant_cols]

# Eliminar df_dask, porque no se va a volver a utilizar
del df
gc.collect()


0

In [6]:
preschool_primary_secondary_education_Mineducacion_part_2.isnull().sum()

,0
anno_inf,0
departamento,5143
cod_dane_municipio,0
municipio,5143
codigo_dane,0
nombre_establecimiento,0
sector,0
calendario,0
nombre_sede,0
zona,0


### Gestionar valores nulos

In [7]:
# Identificar las filas que tienen al menos un valor nulo
nulos = preschool_primary_secondary_education_Mineducacion_part_2[
    preschool_primary_secondary_education_Mineducacion_part_2.isnull().any(axis=1)
]

# Mostrar las primeras filas (debe computarse)
nulos.head()

,anno_inf,departamento,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,especialidad,edad,genero,grupo_etnico,total_matricula
1946,2022,NaN,5380,NaN,205380000050,I. E. ANA EVA ESCOBAR GONZALEZ,OFICIAL,A,I. E. ANA EVA ESCOBAR,Urbana,Fin de Semana,Ciclo 6 Adultos,No aplica,23,Femenino,No aplica,1
2255,2022,NaN,5380,NaN,205380000050,I. E. ANA EVA ESCOBAR GONZALEZ,OFICIAL,A,I. E. ANA EVA ESCOBAR,Urbana,Fin de Semana,Ciclo 6 Adultos,No aplica,24,Femenino,No aplica,3
2358,2022,NaN,5380,NaN,205380000050,I. E. ANA EVA ESCOBAR GONZALEZ,OFICIAL,A,I. E. ANA EVA ESCOBAR,Urbana,Fin de Semana,Ciclo 6 Adultos,No aplica,38,Masculino,No aplica,1
2398,2022,NaN,5380,NaN,205380000050,I. E. ANA EVA ESCOBAR GONZALEZ,OFICIAL,A,I. E. ANA EVA ESCOBAR,Urbana,Fin de Semana,Ciclo 6 Adultos,No aplica,20,Masculino,No aplica,5
2418,2022,NaN,5380,NaN,205380000050,I. E. ANA EVA ESCOBAR GONZALEZ,OFICIAL,A,I. E. ANA EVA ESCOBAR,Urbana,Fin de Semana,Ciclo 6 Adultos,No aplica,21,Femenino,No aplica,3


In [8]:
len(nulos)

5143

Es posible utilizar la columna codigo_dane, que se supone es única para cada establecimiento educativo, para reemplazar los valores nulos de las columnas departamento y municipio, basándote en las filas donde esos valores ya están presentes. Este enfoque se llama imputación por correspondencia o imputación basada en relaciones existentes.

In [9]:
# Crear un mapeo basado en codigo_dane:
codigo_dane_map = preschool_primary_secondary_education_Mineducacion_part_2.dropna(subset=['departamento', 'municipio']).groupby('codigo_dane')[['departamento', 'municipio']].first()

# Identificar las filas con valores nulos en esas tres columnas:
nulos_df = preschool_primary_secondary_education_Mineducacion_part_2[
    preschool_primary_secondary_education_Mineducacion_part_2['departamento'].isnull() |
    preschool_primary_secondary_education_Mineducacion_part_2['municipio'].isnull()
].copy()

# Asegurarse de que las columnas sean de tipo str antes de usar fillna
nulos_df['departamento'] = nulos_df['departamento'].astype(str)
nulos_df['municipio'] = nulos_df['municipio'].astype(str)

# Rellenar los valores nulos utilizando el mapeo:
nulos_df.loc[:, 'departamento'] = nulos_df['codigo_dane'].map(codigo_dane_map['departamento']).fillna(nulos_df['departamento'])
nulos_df.loc[:, 'municipio'] = nulos_df['codigo_dane'].map(codigo_dane_map['municipio']).fillna(nulos_df['municipio'])

# Actualizar las filas del DataFrame original con los valores corregidos:
preschool_primary_secondary_education_Mineducacion_part_2.update(nulos_df)


In [10]:
preschool_primary_secondary_education_Mineducacion_part_2.head()

,anno_inf,departamento,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,especialidad,edad,genero,grupo_etnico,total_matricula
0,2022,VALLE DEL CAUCA,76520,PALMIRA,276520000564,INSTITUCION EDUCATIVA TABLONES,OFICIAL,A,TABLONES,Rural,Fin de Semana,Ciclo 6 Adultos,No aplica,19,Femenino,No aplica,2
1,2022,VALLE DEL CAUCA,76001,CALI,376001028529,COLEGIO SAN IGNACIO DE LOYOLA,NO_OFICIAL,B,COLEGIO SAN IGNACIO DE LOYOLA,Urbana,Mañana,Octavo,No aplica,15,Masculino,No aplica,1
2,2022,VALLE DEL CAUCA,76001,CALI,376001028529,COLEGIO SAN IGNACIO DE LOYOLA,NO_OFICIAL,B,COLEGIO SAN IGNACIO DE LOYOLA,Urbana,Mañana,Octavo,No aplica,14,Femenino,No aplica,2
3,2022,VALLE DEL CAUCA,76001,CALI,376001028529,COLEGIO SAN IGNACIO DE LOYOLA,NO_OFICIAL,B,COLEGIO SAN IGNACIO DE LOYOLA,Urbana,Mañana,Octavo,No aplica,13,Masculino,No aplica,2
4,2022,VALLE DEL CAUCA,76001,CALI,376001029908,LICEO JUVENIL EL RODEO,NO_OFICIAL,A,LICEO JUVENIL EL RODEO,Urbana,Mañana,Octavo,No aplica,13,Femenino,No aplica,3


In [11]:
preschool_primary_secondary_education_Mineducacion_part_2.shape

(5955274, 17)

In [12]:
preschool_primary_secondary_education_Mineducacion_part_2.isnull().sum()

,0
anno_inf,0
departamento,0
cod_dane_municipio,0
municipio,0
codigo_dane,0
nombre_establecimiento,0
sector,0
calendario,0
nombre_sede,0
zona,0


In [13]:
# Llamar al recolector de basura
gc.collect()

0

In [14]:
# Los  nulos que quedaron despues de la imputación por correspondencia o imputación basada en relaciones existentes se eliminan
preschool_primary_secondary_education_Mineducacion_part_2 = preschool_primary_secondary_education_Mineducacion_part_2.dropna(subset=['departamento', 'municipio'])

In [15]:
preschool_primary_secondary_education_Mineducacion_part_2[['departamento', 'municipio']].isnull().sum()

,0
departamento,0
municipio,0


- Verificar anos que cubre la informacion

In [16]:
preschool_primary_secondary_education_Mineducacion_part_2['anno_inf'].unique()

array([2022, 2020, 2021])

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [17]:
categorical_col = preschool_primary_secondary_education_Mineducacion_part_2.select_dtypes(include=['object']).columns.tolist()

In [18]:
# Imprimir categorías únicas para columnas de tipo object
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(preschool_primary_secondary_education_Mineducacion_part_2[column].unique())
    print()

Categorías en la columna 'departamento':
['VALLE DEL CAUCA' 'CUNDINAMARCA' 'BOYACÁ' 'CESAR' 'SANTANDER' 'GUAJIRA'
 'NARIÑO' 'CAPITAL BOGOTÁ, D.C.' 'BOLÍVAR' 'CÓRDOBA' 'ANTIOQUIA' 'nan'
 'CAQUETÁ' 'PUTUMAYO' 'ATLÁNTICO' 'SUCRE' 'META' 'TOLIMA' 'MAGDALENA'
 'CALDAS' 'HUILA' 'CHOCÓ' 'NORTE DE SANTANDER' 'AMAZONAS' 'GUAVIARE'
 'VICHADA' 'CAUCA' 'QUINDIO' 'RISARALDA' 'CASANARE' 'ARAUCA' 'VAUPÉS'
 'GUAINÍA' 'ARCHIPIÉLAGO DE SAN ANDRES,PROVIDENCIA Y SANTA CATALINA']

Categorías en la columna 'cod_dane_municipio':
['76520' '76001' '25290' ... '94887' '94885' '94886']

Categorías en la columna 'municipio':
['PALMIRA' 'CALI' 'FUSAGASUGÁ' ... 'PANA PANA' 'LA GUADALUPE' 'CACAHUAL']

Categorías en la columna 'nombre_establecimiento':
['INSTITUCION EDUCATIVA TABLONES' 'COLEGIO SAN IGNACIO DE LOYOLA'
 'LICEO  JUVENIL  EL  RODEO' ... 'CENTRO EDUCATIVO LA PAULINA'
 'CENTRO EDUCATIVO EL TAGUAL' 'COL MARIO BENEDETTI']

Categorías en la columna 'sector':
['OFICIAL' 'NO_OFICIAL']

Categorías en la columna 

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [19]:
import unicodedata

# Función para eliminar acentos y caracteres especiales
def remove_accents_and_special_chars(input_str):
    if isinstance(input_str, str):  # Verifica si el valor es una cadena
        # Normalizar la cadena a NFKD
        nfkd_form = unicodedata.normalize('NFKD', input_str)

        # Eliminar acentos
        no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

        # Definir caracteres no deseados
        unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

        # Eliminar caracteres no deseados
        cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

        # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
        result = cleaned_str.strip().upper()

        return result
    else:
        # Si no es una cadena (e.g., NaN o None), devolver el valor tal cual
        return input_str

In [20]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    preschool_primary_secondary_education_Mineducacion_part_2.loc[:, col] = preschool_primary_secondary_education_Mineducacion_part_2[col].apply(remove_accents_and_special_chars)


- Codificación de algunas variables categóricas

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codmunicipio"

In [21]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_dane_municipio" en el df preschool_primary_secondary_education_Mineducacion_part_2

In [22]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
preschool_primary_secondary_education_Mineducacion_part_2.loc[:,'cod_dane_municipio'] = preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [23]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 4]


In [24]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_dane_municipio,count,percentage
0,5,5064735,0.850462
1,4,890539,0.149538


In [25]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = preschool_primary_secondary_education_Mineducacion_part_2[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_dane_municipio']])
    print()

Muestra de registros con longitud 5:
  cod_dane_municipio
0              76520
1              76001
2              76001
3              76001
4              76001

Muestra de registros con longitud 4:
     cod_dane_municipio
1940               5360
1941               5360
1945               5360
1946               5380
1948               5360



Nota: De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 8% tiene 5 dígitos (que corresponde con el codigo real que es de 5 dígitos), el 15% tiene solamente 4 dígitos (al parecer el cero a la izquierda de los códigos se suprimió).

- Verificar si los codigo_dane de 4 dígitos corresponden a departamentos que se identifican con el 0 + 1 dígito, para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [26]:
# Filtrar las filas donde 'codigo_dane' tiene 4 dígitos
filtered = preschool_primary_secondary_education_Mineducacion_part_2[preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'].str.len() == 4]

# Obtener el primer dígito de cada código y convertirlo a una serie
first_digit = filtered['cod_dane_municipio'].str[0]
# Obtener las categorías únicas de la columna 'departamento'
first_digit.unique()

array(['5', '8'], dtype=object)

Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane que inicia con un cero + un dígito, 05 y 08 respectivamente.

  - Adicionar un cero a los codigo_dane de 4 dígitos

In [27]:
# Función que agrega un '0' a la izquierda si la longitud del string es 4
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
preschool_primary_secondary_education_Mineducacion_part_2.loc[:,'cod_dane_municipio'] = preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'].apply(add_zero_if_length_4)

In [28]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(preschool_primary_secondary_education_Mineducacion_part_2[preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'].str.len() == 4])

0

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [29]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [30]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

  - Comparar nombres de departamentos en los dataframes

In [31]:
# Comparar listas de departamento
compare_lists(preschool_primary_secondary_education_Mineducacion_part_2['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en preschool_primary_secondary_education_Mineducacion_part_2", "Departamentos dept_mpios_codes")

Departamentos en preschool_primary_secondary_education_Mineducacion_part_2 que no están en Departamentos dept_mpios_codes:
{'GUAJIRA', 'ARCHIPIELAGO DE SAN ANDRESPROVIDENCIA Y SANTA CATALINA', 'NAN', 'CAPITAL BOGOTA D.C.'}


- Ajustamos los nombres de departamento

In [32]:
# Crear el diccionario manualmente
mapeo_departamento = {
    'GUAJIRA': 'LA GUAJIRA',
    'CAPITAL BOGOTA D.C.': 'BOGOTA D.C.',
    'ARCHIPIELAGO DE SAN ANDRESPROVIDENCIA Y SANTA CATALINA': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'
}

# Hacer mapeo para corregir nombres
preschool_primary_secondary_education_Mineducacion_part_2['departamento'] = (
    preschool_primary_secondary_education_Mineducacion_part_2['departamento']
    .map(mapeo_departamento)
    .combine_first(preschool_primary_secondary_education_Mineducacion_part_2['departamento'])  # Mantiene los nombres originales si no hay mapeo
)

- Comprobar si cambio se produjo

In [33]:
# Comparar listas de departamento
compare_lists(preschool_primary_secondary_education_Mineducacion_part_2['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en preschool_primary_secondary_education_Mineducacion_part_2", "Departamentos dept_mpios_codes")

Departamentos en preschool_primary_secondary_education_Mineducacion_part_2 que no están en Departamentos dept_mpios_codes:
{'NAN'}


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [34]:
# Comparar listas de códigos
compare_lists(preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en preschool_primary_secondary_education_Mineducacion_part_2", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en preschool_primary_secondary_education_Mineducacion_part_2 que no están en Códigos de municipios dept_mpios_codes:
set()


In [35]:
# A nonbmre de que municipio aparace el codigo que no tiene coincidencia
preschool_primary_secondary_education_Mineducacion_part_2[preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'] == '94663']

,anno_inf,departamento,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,especialidad,edad,genero,grupo_etnico,total_matricula


Nota: En la bd del proyecto Mapiripana en Guania, figura como vereda de Barrancaminas hacemos el ajuste

In [36]:
# Reemplazamos cod de MAPIRIPANA por BARRANCOMINAS
preschool_primary_secondary_education_Mineducacion_part_2.loc[:, 'cod_dane_municipio'] = preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'].replace({'94663': '94343'})

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [37]:
# Comparar listas de códigos
compare_lists(preschool_primary_secondary_education_Mineducacion_part_2['cod_dane_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en preschool_primary_secondary_education_Mineducacion_part_2", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en preschool_primary_secondary_education_Mineducacion_part_2 que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [38]:
# Eliminar columnas 'departamento', 'municipio' que ya no son necesarias
columns_to_drop = ['departamento', 'municipio']
final_preschool_primary_secondary_education_Mineducacion_part_2 = preschool_primary_secondary_education_Mineducacion_part_2.drop(columns=columns_to_drop)

In [39]:
# Adicionar columna para trazabilidad de la fuente
final_preschool_primary_secondary_education_Mineducacion_part_2['source_id'] = 115

In [40]:
final_preschool_primary_secondary_education_Mineducacion_part_2.columns

Index(['anno_inf', 'cod_dane_municipio', 'codigo_dane',
       'nombre_establecimiento', 'sector', 'calendario', 'nombre_sede', 'zona',
       'tipo_jornada', 'grado', 'especialidad', 'edad', 'genero',
       'grupo_etnico', 'total_matricula', 'source_id'],
      dtype='object')

In [41]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'anno_inf':'year',
    'cod_dane_municipio': 'dane_code',
    'codigo_dane': 'educational_institution_dane_code',
    'nombre_establecimiento': 'educational_institution_name',
    'sector': 'sector',
    'calendario': 'calendar',
    'nombre_sede': 'institution_name',
    'zona': 'area',
    'tipo_jornada': 'shift_type',
    'grado': 'level',
    'especialidad': 'speciallity',
    'edad': 'age',
    'genero': 'gender',
    'grupo_etnico': 'ethnic_group',
    'total_matricula': 'total_enrollment',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_preschool_primary_secondary_education_Mineducacion_part_2.rename(columns=translation_map, inplace=True)

In [42]:
#Estructura final del dataset a integrar a la base de datos
final_preschool_primary_secondary_education_Mineducacion_part_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5955274 entries, 0 to 5955273
Data columns (total 16 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   year                               int64 
 1   dane_code                          object
 2   educational_institution_dane_code  int64 
 3   educational_institution_name       object
 4   sector                             object
 5   calendar                           object
 6   institution_name                   object
 7   area                               object
 8   shift_type                         object
 9   level                              object
 10  speciallity                        object
 11  age                                int64 
 12  gender                             object
 13  ethnic_group                       object
 14  total_enrollment                   int64 
 15  source_id                          int64 
dtypes: int64(5), object(11)
memory usage

## Salvar en archivo csv en el drive

In [43]:
# Guardar en archivos CSV en el drive
final_preschool_primary_secondary_education_Mineducacion_part_2.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/preschool_primary_secondary_education_Mineducacion_part_2.csv', index=False)